In [ ]:
# imports

import pandas as pd
from numpy import dot
from numpy.linalg import norm
import pickle
!pip install openai
import openai

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"
openai.api_key = '' #Insert your openAi token

### 2. Load data

In [5]:
!git clone https://github.com/drndr/genbench_ds.git

Cloning into 'genbench_ds'...
remote: Enumerating objects: 330, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 330 (delta 133), reused 273 (delta 96), pack-reused 0
Receiving objects: 100% (330/330), 8.49 MiB | 8.73 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [ ]:
!pip install datasets

In [ ]:
!python /content/genbench_ds/data/mrr/codesearchnet_go/create_ds_mrr_go.py

Extracting data files: 100% 1/1 [00:18<00:00, 18.19s/it]
Extracting data files: 100% 3/3 [00:09<00:00,  3.23s/it]
Generating train split:  32% 101074/317832 [00:58<01:56, 1867.84 examples/s]

In [ ]:
import json

def read_jsonl(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data
filename = '/content/test_go_mrr.jsonl'
data = read_jsonl(filename)

In [ ]:
len(data)

In [11]:
import json
with open('test_go.json', 'w') as f:
    json.dump(data, f)

In [12]:
import random
from typing import Dict, List
import tqdm
import datasets
import numpy as np
from more_itertools import chunked


def get_dataset_raw(n_distractors) -> Dict[str, datasets.Dataset]:
    raw_datasets: Dict[str, datasets.Dataset] = _load_data_source()
    output: Dict[str, datasets.Dataset] = {}
    random.seed(42)

    for split, dataset in raw_datasets.items():
        if split == "test":
            # Convert dataset to list for easier manipulation
            dataset_list = list(dataset)

            new_data = []

            for idx, item in enumerate(dataset_list):
                new_data.append(item)

                # Create other_items list once and then simply exclude the current item during sampling
                other_items = dataset_list[:idx] + dataset_list[idx+1:]
                random_items = random.sample(other_items, n_distractors)

                input_parts = item["input"].split("[CODESPLIT]")

                for random_item in random_items:
                    random_input_parts = random_item["input"].split("[CODESPLIT]")
                    new_input = input_parts[0] + "[CODESPLIT]" + random_input_parts[1]
                    new_item = {"input": new_input, "target": 0, "target_options": item["target_options"]}
                    new_data.append(new_item)
                print(len(new_data))

            # Convert list back to HuggingFace dataset
            output[split] = datasets.Dataset.from_dict({k: [dic[k] for dic in new_data] for k in new_data[0]})
        else:
            output[split] = dataset

    return output

def _load_data_source() -> Dict[str, datasets.Dataset]:
    return datasets.load_dataset("json", data_files={'test':'/content/test_go.json'})

In [ ]:
n_distractors = 99  # adjust this number as per your requirements
new_datasets = get_dataset_raw(n_distractors)

In [14]:
len(new_datasets['test'])

227900

### 3. Build cache to save embeddings

Before getting embeddings for these articles, let's set up a cache to save the embeddings we generate. In general, it's a good idea to save your embeddings so you can re-use them later. If you don't save them, you'll pay again each time you compute them again.

The cache is a dictionary that maps tuples of `(text, model)` to an embedding, which is a list of floats. The cache is saved as a Python pickle file.

In [15]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "recommendations_embeddings_cache_go.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

Let's check create a dataframe to save the similarities

In [16]:
import pandas as pd
import numpy as np

target = []
for i in range(len(new_datasets['test'])):
  target.append(new_datasets['test'][i]['target'])


# initialize data of lists.
data_ = {'Target': target}

# Create DataFrame
df = pd.DataFrame(data_)
df['Prediciton'] = np.nan
# Print the output.
df

,Target,Prediciton
0,1,NaN
1,0,NaN
2,0,NaN
3,0,NaN
4,0,NaN
...,...,...
227895,0,NaN
227896,0,NaN
227897,0,NaN
227898,0,NaN


Calculate the similarity between the query and the code using Ada's embeddings

In [ ]:
#Go dataset
from sklearn.metrics.pairwise import cosine_similarity

i = 0
while i < len(df):

        input_split = new_datasets['test'][i]['input'].split('[CODESPLIT]')

        x = embedding_from_string(input_split[0])
        y = embedding_from_string(input_split[1])

        df.loc[i, 'Prediciton'] = cosine_similarity([x], [y])[0][0]

        if i % 10000 == 0:
            print(f"Progress: {i}/{len(df)}")
            df.to_csv('/content/drive/MyDrive/CodeInspector/Workshop dataset paper/Results/MRR/MRR_Go_sklearn.csv', index=False)
    i+=1

In [ ]:
df.isnull().values.any()

In [23]:
df.to_csv('MRR_Go_sklearn.csv', index=False)

**Evaluation**

Calculate the MRR results

In [ ]:
!pip install datasets

In [ ]:
import random
from typing import Dict, List
import datasets
import numpy as np
from more_itertools import chunked

def evaluate_predictions(predictions: List[Dict[str, float]]) -> Dict[str, float]:
        """Calculate the MRR score in chunks. One chunk consist of a true comment-code pair and n number of distractors
        This function assumes that the predictions were made and passed onto this function unshuffled.
        The test data is ordered with each true pair followed by n number of distractors
         Args:
             predictions: A list of dictionaries, where each dictionary contains the predicted values for an example.
                          The keys are strings and the values are floats (logit scores or similarity values).
             gold: A HuggingFace `datasets.Dataset` object containing the ground truth data for the task.
             n_distractors:  Number of distractor comment-code pair for each true pair.
                             Must be the same number as in the get_dataset_raw function

         Returns:
             A dictionary containing key-value pairs for the evaluation metric(s) computed on the predicted
             values. The keys are strings representing the name of the evaluation metric and the values are
             floating-point numbers.
        """
        ranks = []

        batched_predictions = chunked(predictions, 99 + 1)

        for batch_idx, predictions in enumerate(batched_predictions):
            correct_score = predictions[0]["score"]
            scores = np.array([prediction["score"] for prediction in predictions])
            rank = np.sum(scores >= correct_score)
            ranks.append(rank)
            if rank == 0:
                print(f"Batch {batch_idx} has a problem. Correct score: {correct_score}. All scores: {scores}")
            ranks.append(rank)
        mean_mrr = np.mean(1.0 / np.array(ranks))

        return {"mean mrr": mean_mrr}

In [ ]:
# Convert your DataFrame's prediction column into the format the function expects
predictions_list = [{"score": score} for score in df['Prediciton'].tolist()]

In [ ]:
# Call the function
n_distractors = 99  # replace this with your actual number of distractors
result = evaluate_predictions(predictions_list)
print(result)

{'mean mrr': 0.8942652125683934}
